In [138]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# Set up environment variables
project_id = 'eighth-arbor-396212'

os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'


In [139]:
def generate_vital_sign_item_codes(vital_sign, conditions):
    conditions_str = " OR ".join([f"LOWER(d.label) LIKE '{condition}'" for condition in conditions])
    cte_sql = f"""
    {vital_sign} AS (
        SELECT *
        FROM `physionet-data.mimiciv_icu.d_items` d
        WHERE {conditions_str}
    )
    """
    return cte_sql

# Define vital signs and their corresponding SQL conditions
vital_sign_conditions = {
    'HeartRate': ["%heart rate%"],
    'BloodPressure': ["%blood pressure%", "%bp%"],
    'RespiratoryRate': ["%respiratory rate%", "%resp rate%"],
    'Temperature': ["%temperature%", "%temp%"],
    'OxygenSaturation': ["%oxygen saturation%", "%spo2%"],
}

vital_item_codes = {vital_sign: generate_vital_sign_item_codes(vital_sign, conditions)
            for vital_sign, conditions in vital_sign_conditions.items()}


In [140]:
def generate_vital_sign_features_cte(vital_sign_name, features=None):
    if features is None:
        features = ['AVG', 'MIN', 'MAX', 'STDDEV', 'VARIANCE', 'PERCENTILE_CONT(0.25)', 'PERCENTILE_CONT(0.75)', 'COUNT']

    aggregate_feature_calculations = []
    for feature in features:
        if "PERCENTILE_CONT" in feature:
            percentile = feature.replace("PERCENTILE_CONT(", "").replace(")", "")
            feature_str = f"APPROX_QUANTILES(valuenum, 100)[SAFE_OFFSET({int(float(percentile) * 100)})] AS percentile_cont_{percentile.replace('.', '_')}{vital_sign_name.lower()}"
        else:
            feature_str = f"{feature}(valuenum) AS {feature.lower()}{vital_sign_name.lower()}"
        aggregate_feature_calculations.append(feature_str)

    aggregate_feature_calculations = ', '.join(aggregate_feature_calculations)


    cte_sql = f"""
    {vital_sign_name}GeneralFeatures AS (
        SELECT DISTINCT subject_id, hadm_id, stay_id,
               {aggregate_feature_calculations}
        FROM `physionet-data.mimiciv_icu.chartevents`
        WHERE itemid IN (SELECT itemid FROM {vital_sign_name})
        GROUP BY subject_id, hadm_id, stay_id
    )
    """

    return cte_sql

vital_signs_general_data = {vital_sign: generate_vital_sign_features_cte(vital_sign)
                              for vital_sign in vital_sign_conditions}

In [141]:
def generate_window_vital_sign_features_cte(vital_sign_name, features=None):
    if features is None:
        features = ['FIRST_VALUE', 'LAST_VALUE']

    window_feature_calculations = ', '.join([
        f"{feature}(valuenum) OVER (PARTITION BY subject_id, hadm_id, stay_id ORDER BY charttime) AS {feature.lower().replace('(','_').replace(')','_').replace('.','_')}{vital_sign_name.lower()}"
        for feature in features
    ])

    rate_of_change_calculation = f"LEAD(valuenum, 1) OVER (PARTITION BY subject_id, hadm_id, stay_id ORDER BY charttime) - valuenum AS rate_of_change_{vital_sign_name.lower()}"

    cte_sql = f"""
    {vital_sign_name}WindowFeatures AS (
        SELECT subject_id, hadm_id, stay_id, charttime, valuenum,
               {window_feature_calculations},
               {rate_of_change_calculation}
        FROM `physionet-data.mimiciv_icu.chartevents`
        WHERE itemid IN (SELECT itemid FROM {vital_sign_name})
        WINDOW w AS (PARTITION BY subject_id, hadm_id, stay_id ORDER BY charttime)
    )
    """

    return cte_sql

generate_window_vital_signs_dict = {}

for vital_sign in vital_sign_conditions:
    generate_window_vital_signs_dict[vital_sign] = generate_window_vital_sign_features_cte(vital_sign)

In [142]:
def time_difference_data(vital_sign_table, vital_sign_column):
    return f"""
    {vital_sign_table}TimeDifferenceData AS (
        SELECT
            subject_id,
            hadm_id,
            stay_id,
            TIMESTAMP_DIFF(lead(charttime) OVER w, charttime, SECOND) AS time_interval_seconds,
            {vital_sign_column} AS {vital_sign_table.lower()}_rate
        FROM {vital_sign_table}WindowFeatures
        WINDOW w AS (PARTITION BY subject_id, hadm_id, stay_id ORDER BY charttime)
    )"""

time_difference_dict = {}

for vital_sign in vital_sign_conditions:
    time_difference_dict[vital_sign] = time_difference_data(vital_sign, "valuenum")

In [143]:
def vital_signs_clinical_features(lower_bound, upper_bound, vital_sign, condition_low, condition_high):
    return f"""
    {vital_sign}ClinicalFeatures AS (
        SELECT
            subject_id,
            hadm_id,
            stay_id,
            SUM(CASE WHEN {vital_sign.lower()}_rate > {upper_bound} THEN time_interval_seconds ELSE 0 END) AS time_with_{condition_high},
            SUM(CASE WHEN {vital_sign.lower()}_rate < {lower_bound} THEN time_interval_seconds ELSE 0 END) AS time_with_{condition_low},
            COUNTIF({vital_sign.lower()}_rate > {upper_bound}) AS episodes_of_{condition_high},
            COUNTIF({vital_sign.lower()}_rate < {lower_bound}) AS episodes_of_{condition_low}
        FROM {vital_sign}TimeDifferenceData
        GROUP BY subject_id, hadm_id, stay_id
    )"""


# Vital signs with their respective lower and upper bounds
# Hypotension and Hypertension for BloodPressure
# Bradypnea and Tachypnea for RespiratoryRate
# Hypoxemia and Hyperoxia for OxygenSaturation
# Bradycardia and Tachycardia for HeartRate
# Hypothermia and Hyperthermia for Temperature
vital_signs = {
    'BloodPressure': {'lower': 60, 'upper': 180, 'conditions': 'Hypotension, Hypertension'},
    'RespiratoryRate': {'lower': 10, 'upper': 30, 'conditions': 'Bradypnea, Tachypnea'},
    'OxygenSaturation': {'lower': 90, 'upper': 100, 'conditions': 'Hypoxemia, Hyperoxia'},
    'HeartRate': {'lower': 50, 'upper': 100, 'conditions': 'Bradycardia, Tachycardia'},
    'Temperature': {'lower': 35, 'upper': 38, 'conditions': 'Hypothermia, Hyperthermia'},
}

# Create a dictionary to store the Clinical Features CTEs
clinical_features_dict = {}

# Populate the dictionary
for vital_sign, bounds in vital_signs.items():
    lower_bound = bounds['lower']
    upper_bound = bounds['upper']
    condition_low, condition_high = bounds['conditions'].split(', ')
    clinical_features_cte = vital_signs_clinical_features(lower_bound, upper_bound, vital_sign, condition_low, condition_high)
    clinical_features_dict[vital_sign] = clinical_features_cte


In [144]:
def join_ctes(cte_dict):
    """
    Joins multiple CTEs into a single SQL string.

    Parameters:
    cte_dict (dict): A dictionary where keys are CTE names and values are CTE SQL strings.

    Returns:
    str: A single SQL string containing all the CTEs.
    """
    cte_list = list(cte_dict.values())

    # Add 'WITH' to the first CTE
    cte_list[0] = "WITH " + cte_list[0].strip()

    # Add trailing comma to all except the last CTE
    cte_list[:-1] = [cte.strip() + "," for cte in cte_list[:-1]]

    # Concatenate all the CTEs
    joined_ctes = " ".join(cte_list)

    return joined_ctes

cte_dict = {}
# Example usage
for vital_sign in vital_sign_conditions:
    cte_dict[vital_sign] = {}
    cte_dict[vital_sign][vital_sign] = vital_item_codes[vital_sign]
    cte_dict[vital_sign][f"{vital_sign}GeneralFeatures"] = vital_signs_general_data[vital_sign]
    cte_dict[vital_sign][f"{vital_sign}WindowFeatures"] = generate_window_vital_signs_dict[vital_sign]
    cte_dict[vital_sign][f"{vital_sign}TimeDifferenceData"] = time_difference_dict[vital_sign]
    cte_dict[vital_sign][f"{vital_sign}ClinicalFeatures"] = clinical_features_dict[vital_sign]

joined_ctes = {}
for vital_sign in vital_sign_conditions:
    joined_ctes[vital_sign] = join_ctes(cte_dict[vital_sign])


In [145]:
def vital_signs_select_all(vital_sign_name, ctes, limit=True):

    sql_query = f"""
    {ctes}
    SELECT *
    FROM {vital_sign_name}GeneralFeatures
    JOIN {vital_sign_name}ClinicalFeatures
    USING (subject_id, hadm_id, stay_id)
    {'LIMIT 1000' if limit else ''};
    """

    return sql_query
# 722758

vital_signs_queries = {}

for vital_sign in vital_sign_conditions:
    vital_signs_queries[vital_sign] = vital_signs_select_all(vital_sign, joined_ctes[vital_sign])
    print(vital_signs_queries[vital_sign])


    WITH HeartRate AS (
        SELECT *
        FROM `physionet-data.mimiciv_icu.d_items` d
        WHERE LOWER(d.label) LIKE '%heart rate%'
    ), HeartRateGeneralFeatures AS (
        SELECT DISTINCT subject_id, hadm_id, stay_id,
               AVG(valuenum) AS avgheartrate, MIN(valuenum) AS minheartrate, MAX(valuenum) AS maxheartrate, STDDEV(valuenum) AS stddevheartrate, VARIANCE(valuenum) AS varianceheartrate, APPROX_QUANTILES(valuenum, 100)[SAFE_OFFSET(25)] AS percentile_cont_0_25heartrate, APPROX_QUANTILES(valuenum, 100)[SAFE_OFFSET(75)] AS percentile_cont_0_75heartrate, COUNT(valuenum) AS countheartrate
        FROM `physionet-data.mimiciv_icu.chartevents`
        WHERE itemid IN (SELECT itemid FROM HeartRate)
        GROUP BY subject_id, hadm_id, stay_id
    ), HeartRateWindowFeatures AS (
        SELECT subject_id, hadm_id, stay_id, charttime, valuenum,
               FIRST_VALUE(valuenum) OVER (PARTITION BY subject_id, hadm_id, stay_id ORDER BY charttime) AS first_valuehear

In [146]:
run_query(vital_signs_queries["HeartRate"])


,subject_id,hadm_id,stay_id,avgheartrate,minheartrate,maxheartrate,stddevheartrate,varianceheartrate,percentile_cont_0_25heartrate,percentile_cont_0_75heartrate,countheartrate,time_with_Tachycardia,time_with_Bradycardia,episodes_of_Tachycardia,episodes_of_Bradycardia
0,18336985,27138595,39529590,74.370690,49.0,130.0,24.647614,607.504873,58.0,77.0,116,23460,7200,22,1
1,18388605,25487625,37663999,94.166667,60.0,120.0,14.090524,198.542857,85.0,104.0,36,37800,0,14,0
2,18932705,22245247,31653552,72.058824,40.0,130.0,19.274825,371.518876,60.0,80.0,68,5280,7200,5,4
3,19116514,28456839,38758840,92.181818,50.0,150.0,17.078999,291.692216,82.0,102.0,154,65940,0,42,0
4,18973018,24553597,39142568,81.421533,42.0,130.0,14.914406,222.439499,72.0,88.0,1096,137280,960,87,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10706411,22795641,31395458,98.980456,50.0,140.0,17.038968,290.326414,93.0,108.0,307,346200,0,151,0
996,12686410,24762560,37206875,100.246944,50.0,130.0,18.702132,349.769752,94.0,112.0,409,740880,0,253,0
997,14220073,24376515,32568586,83.390244,50.0,130.0,22.621757,511.743902,76.0,85.0,41,1140,0,6,0
998,11739489,21593242,31527133,71.255754,40.0,130.0,21.348298,455.749807,60.0,72.0,391,86460,0,48,3


In [148]:
run_query(vital_signs_queries["BloodPressure"])


,subject_id,hadm_id,stay_id,avgbloodpressure,minbloodpressure,maxbloodpressure,stddevbloodpressure,variancebloodpressure,percentile_cont_0_25bloodpressure,percentile_cont_0_75bloodpressure,countbloodpressure,time_with_Hypertension,time_with_Hypotension,episodes_of_Hypertension,episodes_of_Hypotension
0,12329855,26570623,39965206,105.026871,32.0,229.0,31.257295,977.018484,81.0,134.0,1563,180,10980,2,21
1,12389739,28561509,38913055,100.294239,44.0,187.0,34.424665,1185.057575,72.0,127.0,486,0,16140,3,30
2,17832362,21341440,38017251,80.693333,38.0,150.0,27.835922,774.838571,58.0,102.0,225,0,44400,0,63
3,16447217,20354658,32410750,93.664399,32.0,180.0,36.004556,1296.328025,63.0,122.0,441,0,96360,0,82
4,19995790,22970553,34995866,86.284483,46.0,153.0,26.330939,693.318366,65.0,110.0,116,0,3900,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12033684,28693924,36477794,93.550562,50.0,160.0,30.727176,944.159346,70.0,110.0,89,0,0,0,10
996,13438658,26187111,36920085,93.340909,49.0,160.0,29.398579,864.276427,69.0,119.0,44,0,18000,0,6
997,14149589,20398794,30243128,70.408163,37.0,160.0,25.701862,660.585686,51.0,87.0,147,0,97440,0,71
998,12589553,29404026,38169695,90.403846,48.0,160.0,31.595644,998.284691,63.0,114.0,52,0,14340,0,5


In [149]:
run_query(vital_signs_queries["RespiratoryRate"])

,subject_id,hadm_id,stay_id,avgrespiratoryrate,minrespiratoryrate,maxrespiratoryrate,stddevrespiratoryrate,variancerespiratoryrate,percentile_cont_0_25respiratoryrate,percentile_cont_0_75respiratoryrate,countrespiratoryrate,time_with_Tachypnea,time_with_Bradypnea,episodes_of_Tachypnea,episodes_of_Bradypnea
0,18756147,21170807,34900611,22.715580,0.0,55.0,7.879347,62.084113,18.0,28.0,552,312180,19380,91,22
1,13637699,29612379,33214455,18.502982,0.0,43.0,6.856996,47.018399,16.0,22.0,1006,65280,68340,28,75
2,10206590,26927205,35881376,20.688136,0.0,50.0,5.984478,35.813974,17.0,24.0,295,30480,3600,11,2
3,19863368,21372089,32877126,22.284360,0.0,56.0,9.577698,91.732302,19.0,30.0,633,39480,66060,25,77
4,19487171,23826727,32959094,21.561151,0.0,33.0,8.158853,66.566886,22.0,26.0,139,3600,16500,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11062873,26321670,38381060,26.511278,0.0,35.0,6.249384,39.054796,24.0,30.0,133,79200,4080,29,3
996,13607129,28605351,37086280,16.600000,0.0,38.0,6.758850,45.682051,14.0,19.0,40,3600,0,2,2
997,13588636,26266616,31111494,25.403361,0.0,54.0,5.459199,29.802858,22.0,28.0,238,119040,3360,36,2
998,10803096,28708257,33895812,19.120879,0.0,28.0,4.404127,19.396337,16.0,22.0,91,0,0,0,1


In [150]:
run_query(vital_signs_queries["Temperature"])

,subject_id,hadm_id,stay_id,avgtemperature,mintemperature,maxtemperature,stddevtemperature,variancetemperature,percentile_cont_0_25temperature,percentile_cont_0_75temperature,counttemperature,time_with_Hyperthermia,time_with_Hypothermia,episodes_of_Hyperthermia,episodes_of_Hypothermia
0,18813854,20872284,35057631,98.672727,97.6,99.6,0.618209,0.382182,98.1,99.1,11,54600,0,11,0
1,10151556,20815067,31701999,57.994186,0.0,377.0,35.897408,1288.623912,37.0,98.2,774,962760,43800,315,69
2,18357680,24032983,37800448,97.592308,96.0,98.2,0.567834,0.322436,97.5,97.9,13,46500,0,13,0
3,10282931,23897542,38921444,98.654545,96.2,100.4,1.326170,1.758727,98.4,99.4,11,0,0,11,0
4,16686301,22191679,31801765,73.434783,37.0,103.2,30.808113,949.139852,37.0,99.0,207,347520,0,121,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14019847,22565753,39138205,98.762791,97.7,100.9,0.768741,0.590963,98.3,99.2,43,219420,0,43,0
996,10968074,27218193,34428342,100.103846,97.8,103.4,1.689584,2.854695,98.7,101.4,156,592440,0,156,0
997,11839007,21724548,31552848,98.968182,97.6,100.6,0.825421,0.681320,98.4,99.6,22,129960,0,22,0
998,15049054,23597206,33211957,99.204167,98.1,100.3,0.427815,0.183025,98.9,99.3,24,64800,0,24,0


In [151]:
run_query(vital_signs_queries["OxygenSaturation"])

,subject_id,hadm_id,stay_id,avgoxygensaturation,minoxygensaturation,maxoxygensaturation,stddevoxygensaturation,varianceoxygensaturation,percentile_cont_0_25oxygensaturation,percentile_cont_0_75oxygensaturation,countoxygensaturation,time_with_Hyperoxia,time_with_Hypoxemia,episodes_of_Hyperoxia,episodes_of_Hypoxemia
0,15789201,24078757,38180152,85.067308,85.0,92.0,0.686406,0.471154,85.0,85.0,104,0,3320280,0,103
1,16475908,21066762,38813042,85.300000,85.0,88.0,0.948683,0.900000,85.0,85.0,10,0,353940,0,10
2,17898246,21292850,37614062,85.000000,85.0,85.0,0.000000,0.000000,85.0,85.0,32,0,978900,0,32
3,16093254,21076930,32021671,85.000000,85.0,85.0,0.000000,0.000000,85.0,85.0,12,0,450000,0,12
4,19360045,22279720,38200431,85.750000,85.0,88.0,1.500000,2.250000,85.0,85.0,4,0,107640,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15292114,24875769,32093246,88.000000,88.0,88.0,0.000000,0.000000,88.0,88.0,6,0,70260,0,6
996,14747794,21912473,38934901,83.105263,1.0,88.0,19.893870,395.766082,87.0,88.0,19,0,553440,0,19
997,10745469,22726387,36646192,73.571429,2.0,88.0,31.579831,997.285714,85.0,85.0,7,0,198660,0,7
998,11390883,25317413,39793803,87.000000,85.0,90.0,1.655518,2.740741,85.0,88.0,28,0,914400,0,26
